# 크롤링 실습

In [15]:
# 필요한 라이브러리 다운로드
# 이미 설치되어 있으면 실행하지 않으셔도 됩니다

!pip3 install requests
!pip3 install bs4
!pip3 install selenium

     |████████████████████████████████| 127 kB 27.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.0 requires urllib3[secure]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


  Using cached urllib3-1.26.8-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import re
import requests
from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
cine21_url = 'http://www.cine21.com/rank/person/'
res = requests.get(cine21_url)
print(res.text) # get으로 할 경우 배우 리스트가 응답으로 반환되지 않음(사이트마다 다름)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
	<meta property="fb:pages" content="1641311652800771" />
	<meta name="google-site-verification" content="vTM0gmeRzJwn1MIM1LMSp3cxP_SaBzch1ziRY255RHw" />
	<meta name="google-site-verification" content="5yOe6b_e_3rr7vNDwgXJw_8wLZQGx4lJ_V48KNPrqkA" />
	<meta name="naver-site-verification" content="20defde86fc4464f2693891567a98905bd0a60d1" />
	<meta name='dailymotion-domain-verification' content='dmds9ks357rhqvdnk' />
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<title>씨네21</title>
	<link rel="stylesheet" type="text/css" media="all" href="/inc/www/css/default1.css" />
	<link rel="stylesheet" type="text/css" media="all" href="/inc/www/css/content1.css" />
	<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.0/jquery.min.js"></scr

### 크롤링 주소 요청
dictionary를 이용하여 form data 생성   
request모듈의 post 함수를 이용하여 요청 전송

- 첫 번째 인자: 요청 url
- 두 번째 인자: form data

In [26]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2022-01'
post_data['gender'] = 'all'
result = []

for i in range(1, 11):
  post_data['page'] = i
  res = requests.post(cine21_url, data=post_data)
  soup = BeautifulSoup(res.text, 'html.parser')

  actors_info_list = list()
  actors = soup.select('li.people_li div.name')
  actors_num_info = soup.select('ul.num_info li strong')
  actors_mov_list = soup.select('ul.mov_list')
  actors_grade = soup.select('li.people_li span.grade ')

  for index, actor in enumerate(actors):
      actor_info_dict = dict()
      actor_name = re.sub('\(\w*\)', '', actor.text)
      actor_info_dict['이름'] = actor_name
      actor_info_dict['흥행지수'] = actors_num_info[index].text
      actor_mov_list = actors_mov_list[index].select('li a span')
      actor_info_dict['출연영화'] = list()
      for i in actor_mov_list:        
        actor_info_dict['출연영화'].append(i.text)
      actor_info_dict['랭킹'] = actors_grade[index].text

      actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
      response_actor = requests.get(actor_link)
      soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
      default_info = soup_actor.select_one('ul.default_info')
      actor_details = default_info.select('li')
      
      for actor_detail in actor_details:
          actor_detail_key = actor_detail.select_one('span.tit').text
          actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
          actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
          actor_info_dict[actor_detail_key] = actor_detail_value
          
      actors_info_list.append(actor_info_dict)
  
  result.append(actors_info_list)
  # print(actors_info_list)
result


[[{'다른 이름': '김하늘',
   '랭킹': '1',
   '생년월일': '1990-02-21',
   '성별': '남',
   '신장/체중': '181cm, 70kg',
   '이름': '강하늘',
   '직업': '배우',
   '출연영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
   '학교': '중앙대학교 연극학과',
   '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
   '흥행지수': '79,392'},
  {'랭킹': '2',
   '생년월일': '1987-02-22',
   '성별': '여',
   '신장/체중': '170cm',
   '이름': '한효주',
   '직업': '배우',
   '출연영화': ['해적: 도깨비 깃발'],
   '취미': '영화감상',
   '학교': '동국대학교 연극영화',
   '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
   '흥행지수': '67,602'},
  {'랭킹': '3',
   '생년월일': '1985-07-14',
   '성별': '남',
   '소속사': '킹콩엔터테인먼트',
   '신장/체중': '190cm',
   '이름': '이광수',
   '직업': '배우',
   '출연영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
   '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
   '흥행지수': '59,677'},
  {'다른 이름': 'Kwon Sang Woo',
   '랭킹': '4',
   '생년월일': '1976-08-05',
   '성별': '남',
   '소속사': '벨액터스 엔터테인먼트',
   '신장/체중': '183cm, 72kg',
   '이름': '권상우',
   '직업': '배우',
   '출연영화': ['해적: 도깨비 깃발'],
   '취미': '